In [1]:
# use beautiful soup code test
# put in functions for the processes (use class? class could enable for it to run constantly across pages calling itself and then return final dataset)
# end result: dataframe on title and descriptions of the flats which can be exported

In [2]:
# to tackle next:
# write scrappers for other websites (booking next priority) to compare descriptions
# then apply use/test case with one or two cities (one Luxembourg) for all four websites and compare descriptions --> requires rudimentary comparison algorithm
# when scrappers for all websites running improve some details like:
# write function testing the city links before def init? would have to put all the crawling in extra function?
# have all the print statements be exported in txt for each city as a log which then could be searched?
# descriptions sometimes not automatically translated but put in original language and translated to english in one description block --> how solve only getting english one? ignore?
# make crawler run several times, always adding the rooms not yet in the data
# write function within class to create a dataset from the object characteristics
# crawl other things like title, more info, pictures (pictures cannot be part of dataset but maybe exported seperately into predefined data folder structure?)
# write the crawler very general to be able to crawl all webpages? (could do with if statements)
    # would complicate code, having four separate scrappers for each website easier to maintain

In [3]:
# improvements to be implemented:
# need to demojize titles as well --> SOLVED
# extracted cities have a comma at the end --> SOLVED
# never extracts descriptions --> SOLVED
# descriptions still have the |, need to be cleaned more --> SOLVED
# standardize type? --> if includes room, append room, if not, append apartement --> SOLVED

In [4]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import emoji
import re
import pytz
from requests_html import HTMLSession
import spacy
import pandas

/home/fbohm/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
nlp = spacy.load("en_core_web_trf")
city_list = ['Luxembourg--Luxemburg', 'Munich--Germany', 'Paris--France', 'Barcelona--Spain']
main_url = 'https://www.airbnb.com'

In [15]:
class airbnb_data:
    def __init__ (self, city_list, main_url):
        #set up basics
        self.cities_url = {}
        self.number_properties = {}
        self.extraction_date = {}
        self.rooms = {}
        self.description = {}
        self.host = {}
        self.address = {}
        self.street = {} #always empty
        self.district = {}
        self.postal_code = {} #always empty
        self.city = {}
        self.country = {}
        self.title = {}
        self.type = {}
        self.index = {}
        self.platform = {}
        
        for city in city_list:
            url = f'{main_url}/s/{city}/homes'
            
            self.cities_url[city] = [url]
            self.number_properties[city] = []
            self.extraction_date[city] = []
            self.rooms[city] = []
            self.description[city] = []
            self.host[city] = []
            self.address[city] = []
            self.street[city] = []
            self.district[city] = []
            self.postal_code[city] = []
            self.city[city] = []
            self.country[city] = []
            self.title[city] = []
            self.type[city] = []
            self.index[city] = []
            self.platform[city] = []
            
        print('finished basic setup\n')
        
    def DataCollection (self, city_list, log_name):
        log = open(log_name, 'a')

        for city in city_list:
            url = self.cities_url[city][0]
            request = requests.get(url)
            soup = BeautifulSoup(request.content, 'html.parser')
            amount_prop = soup.find(class_ = '_le6wlg')
            amount = amount_prop.get_text(strip = True).split()
            if len(amount) == 3: #meaning the amount of stays at the top of the page says "over 1000 stays" instead of "756 stays"
                self.number_properties[city].append(1001) #1001 as keyword for more than 1000 stays
            else:
                self.number_properties[city].append(int(amount[0]))
            
            self.extraction_date[city].append(new_date) 

            log.write('\nworking on city: ' + city + '\n')
            print('working on city: ' + city + '\n')
            pagecounter = 0
            totalcounter = 0
            desccounter = 0
            start = time.time()
            offset = 20
            #access pages and crawl room links
            start_pages = time.time()
            for site in self.cities_url[city]:
                start_page = time.time()
                log.write('\ntrying to extract room links from: ' + site + '\n')
                sub = '/room'
                request = requests.get(site)
                soup = BeautifulSoup(request.content, 'html.parser')
                links = []
                for a in soup.find_all(href=True):
                    link = a['href']
                    res = link.find(sub)
                    if res != -1:
                        cut = link.find('?')
                        final = main_url + link[:cut]
                        if final not in self.rooms[city] and final not in links:
                            links.append(final)
                if len(links) > 0:
                    pagecounter += 1
                    for element in links:
                        self.rooms[city].append(element)
                    totalcounter += len(links)
                    log.write('\tadded a total of ' + str(len(links)) + ' room URLs\n')
                    log.write('\tcrawled a total of ' + str(pagecounter) + ' pages so far\n')
                    self.cities_url[city].append(url + '?items_offset=' + str(offset))
                    offset += 20
                else:
                    del self.cities_url[city][-1]
                    log.write('\tabove page did not yield any new room links, meaning we have reached the last page\n')
                    log.write('\nreached the final page at ' + str(pagecounter) + ' and added a total of ' + str(totalcounter) + ' room URLs\n')
                    break

                log.write('\tdone with extracting room URLs within ' + str(time.time()-start_page) + ' seconds\n')

            log.write('done with all page URLs within ' + str(time.time()-start_pages) + ' seconds\n')
            print('done with all page URLs within ' + str(time.time()-start_pages) + ' seconds\n')
            log.write('\twe expected a total of ' + str(self.number_properties[city][0]) + ' URLs\n')
            if self.number_properties[city][0] - totalcounter > 0:
                log.write('\twe have crawled less room URLs than expected with a difference of: ' + str(self.number_properties[city][0] - totalcounter) + '\n')
            elif self.number_properties[city][0] - totalcounter == 0:
                log.write('\twe have no difference between the expected and the actually crawled room URLs\n')
            elif self.number_properties[city][0] - totalcounter < 0:
                log.write('\twe crawled more room URLs than expected with a difference of: ' + str(totalcounter - self.number_properties[city][0]) + '\n')

            #extract room details
            descrawl_time = time.time()
            for room in self.rooms[city]:

                room_time = time.time()
                log.write('\ndoing accommodation: '+ room + '\n')
                request = requests.get(room)
                soup = BeautifulSoup(request.content, 'html.parser')
                content = soup.find_all(class_ = 'c1yo0219 dir dir-ltr')

                self.postal_code[city].append('NaN')
                self.street[city].append('NaN')
                self.index[city].append(desccounter)
                self.platform[city].append(10)

                log.write('\tdoing accommodation description\n')
                hint_desc1 = 'Every booking includes free protection'
                hint_desc2 = 'Some info has been automatically translated'
                hint_nodesc1 = "Where you’ll be"
                hint_nodesc2 = 'What this place offers'
                hint_nodesc3 = "Where you'll sleep"
                for i in range(6,len(content)):
                    text_desc = content[i].get_text()
                    if hint_desc1 in text_desc:
                        if hint_nodesc1 in content[i+1].get_text(' ') or hint_nodesc2 in content[i+1].get_text(' ') or hint_nodesc3 in content[i+1].get_text(' '):
                            self.description[city].append('NaN')
                        elif hint_desc2 in content[i+1].get_text(' '):
                            desc = content[i+2].get_text(' ').lower()
                            desc.replace('|', ' ')
                            desc.replace('show more', '')
                            desc.replace('\n', ' ')
                            desc_noemo = emoji.demojize(desc)
                            desc_fin = re.sub(r':[a-z_]+:','', desc_noemo)
                            clean = " ".join(desc_fin.split())
                            self.description[city].append(clean)
                        else:
                            desc = content[i+1].get_text('|').lower()
                            desc.replace('|', ' ')
                            desc.replace('show more', '')
                            desc.replace('\n', ' ')
                            desc_noemo = emoji.demojize(desc)
                            desc_fin = re.sub(r':[a-z_]+:','', desc_noemo)
                            clean = " ".join(desc_fin.split())
                            self.description[city].append(clean)
                log.write('\taccommodation description done\n')

                log.write('\tdoing accommodation host and type\n')
                try:
                    subheader = content[6].get_text('|')
                    cut = subheader.find('|')
                    type_host = subheader[:cut].lower().split()
                    pos_host = type_host.index('by')
                    host = ' '.join(type_host[pos_host + 1:]).replace('\n', ' ')
                    type_ = ' '.join(type_host[0:2]).replace('\n', ' ')
                    self.host[city].append(host)
                    if 'room' in type_:
                        self.type[city].append('room')
                    elif type_ == 'entire apartement' or type_ == 'entire rental':
                        self.type[city].append('apartment')
                    elif 'entire' in type_:
                        type_split = type_.split(' ')
                        self.type[city].append(type_split[-1])
                    else:
                        self.type[city].append(type_)
                except IndexError:
                    print('\n\tIndex Error encountered')
                    print(content)
                    self.host[city].append('error during extraction')
                    self.type[city].append('error during extraction')
                log.write('\taccommodation host and type done\n')  

                log.write('\tdoing accommodation title\n')
                header = content[1].get_text('|')
                cut = header.find('|')
                title = header[:cut].lower().replace('\n', ' ')
                title_noemo = emoji.demojize(title)
                title_fin = re.sub(r':[a-z_]+:','', title_noemo)
                self.title[city].append(title_fin)
                log.write('\taccommodation title done\n')

                log.write('\tdoing accommodation address\n')
                hint_loc = "Where you’ll be"
                pos = 0
                for i in range(6,len(content)):
                    test = content[i].get_text('|')
                    if hint_loc in test:
                        pos = i
                
                if pos != 0:
                    text_add = content[pos].get_text('|')
                    loc = text_add.split('|')
                    loc_ = loc[1].lower().replace('\n', ' ')
                    self.address[city].append(loc_)

                    if len(loc) == 2:
                        self.district[city].append('NaN')
                    else:
                        output = ''
                        loc_desc = loc[2:-1]
                        for part in loc_desc:
                            output += part
                        doc = nlp(output)
                        ents = []
                        for ent in doc.ents:
                            if str(ent).lower() not in loc_:
                                if ent.label_ == 'GPE' or ent.label_ == 'LOC':
                                    ents.append(str(ent).lower())
                        if ents == []:
                            self.district[city].append('NaN')
                        else:
                            counts = []
                            for ent in ents:
                                ent_c = output.count(ent)
                                counts.append(ent_c)
                            pos = counts.index(max(counts))
                            self.district[city].append(ents[pos])

                    loc_split = loc_.split(',')
                    if len(loc_split) == 2:
                        self.city[city].append(loc_split[0].strip(' ').replace('\n', ' '))
                        self.country[city].append(loc_split[1].strip(' ').replace('\n', ' '))
                    elif len(loc_split) == 3:
                        self.city[city].append(loc_split[0].strip(' ').replace('\n', ' '))
                        self.country[city].append(loc_split[2].strip(' ').replace('\n', ' '))
                    else:
                        self.city[city].append('error during extraction')
                        self.country[city].append('error during extraction')
                else:
                    self.address[city].append('error during extraction')
                    self.district[city].append('error during extraction')
                    self.city[city].append('error during extraction')
                    self.country[city].append('error during extraction')

                log.write('\taccommodation address done\n')

                desccounter += 1
                log.write('\tadded description for room within ' + str(time.time()-room_time) + ' seconds\n')
                log.write('\tdescriptions scrapped: ' + str(desccounter) + '\n')

            log.write('\ndone scrapping room descriptions with a total of '+ str(desccounter) + ' within ' + str(time.time()-descrawl_time) + ' seconds\n')

            log.write('done with city: ' + city + ' within ' + str((time.time()-start)/60) + ' minutes\n')
            
        
            print('done with city: ' + city + ' within ' + str((time.time()-start)/60) + ' minutes\n')
            print('compare the lengths of the relevant data dictionaries, they should be identical:')
            print('\tlength of self.rooms', len(self.rooms[city]))
            print('\tlength of self.description', len(self.description[city]))
            print('\tlength of self.host', len(self.host[city]))
            print('\tlength of self.address', len(self.address[city]))
            print('\tlength of self.street', len(self.street[city]))
            print('\tlength of self.district', len(self.district[city]))
            print('\tlength of self.postal_code', len(self.postal_code[city]))
            print('\tlength of self.city', len(self.city[city]))
            print('\tlength of self.country', len(self.country[city]))
            print('\tlength of self.title', len(self.title[city]))
            print('\tlength of self.type', len(self.type[city]))
            print('\tlength of self.index', len(self.index[city]))
            print('\tlength of self.platform', len(self.platform[city]))

            print('\nand for further checks, the other to dictionaries with the metadata:')
            print('\tpage links:', self.cities_url)
            print('\tproperty numbers:', self.number_properties)
            print('\textraction date:', self.extraction_date)
        
        log.close()
        
    def dataset (self, csv_name):
        self.df_dic = {'index': [], 'platform': [], 'room': [], 'title': [], 'type': [], 'desc': [], 'host': [], 'address': [], 
                  'street': [], 'district': [], 'postal_code': [], 'city': [], 'country': []}
        for city in city_list:
            for element in self.rooms[city]:
                self.df_dic['room'].append(element)
            for element in self.description[city]:
                self.df_dic['desc'].append(element)
            for element in self.host[city]:
                self.df_dic['host'].append(element)
            for element in self.address[city]:
                self.df_dic['address'].append(element)
            for element in self.street[city]:
                self.df_dic['street'].append(element)
            for element in self.district[city]:
                self.df_dic['district'].append(element)
            for element in self.postal_code[city]:
                self.df_dic['postal_code'].append(element)
            for element in self.city[city]:
                self.df_dic['city'].append(element)
            for element in self.country[city]:
                self.df_dic['country'].append(element)
            for element in self.title[city]:
                self.df_dic['title'].append(element)
            for element in self.type[city]:
                self.df_dic['type'].append(element)
            for element in self.index[city]:
                self.df_dic['index'].append(element)
            for element in self.platform[city]:
                self.df_dic['platform'].append(element)

        df = pandas.DataFrame.from_dict(self.df_dic)
        df.to_csv(csv_name, index = False, encoding = 'utf-8')
        return df

In [16]:
date = datetime.now(pytz.timezone('Europe/Berlin'))
new_date = f'{date.day}-{date.month}-{date.year}_{date.hour}:{date.minute}'
obj = airbnb_data(city_list,main_url)

finished basic setup



In [17]:
obj.DataCollection(city_list, 'log_airbnb_' + str(new_date) + '.txt')

working on city: Luxembourg--Luxemburg

done with all page URLs within 30.79459834098816 seconds

done with city: Luxembourg--Luxemburg within 5.965430327256521 minutes

compare the lengths of the relevant data dictionaries, they should be identical:
	length of self.rooms 269
	length of self.description 269
	length of self.host 269
	length of self.address 269
	length of self.street 269
	length of self.district 269
	length of self.postal_code 269
	length of self.city 269
	length of self.country 269
	length of self.title 269
	length of self.type 269
	length of self.index 269
	length of self.platform 269

and for further checks, the other to dictionaries with the metadata:
	page links: {'Luxembourg--Luxemburg': ['https://www.airbnb.com/s/Luxembourg--Luxemburg/homes', 'https://www.airbnb.com/s/Luxembourg--Luxemburg/homes?items_offset=20', 'https://www.airbnb.com/s/Luxembourg--Luxemburg/homes?items_offset=40', 'https://www.airbnb.com/s/Luxembourg--Luxemburg/homes?items_offset=60', 'https://

In [ ]:
airbnb_df = obj.dataset('airbnb_' + str(new_date) + '.csv')

### **Code dump and tries**

In [64]:
rooms = []
for site in obj.cities_url['Luxembourg--Luxemburg']:
    
    html = requests.get(site)
    soup = BeautifulSoup(html.content, 'html.parser')
    room_extract = soup.find_all(class_='lwm61be dir dir-ltr')

    room_hrefs = []
    for part in room_extract:
        link = part['href']
        room_hrefs.append(link)

    for index in range(len(room_hrefs)):
        room_link = room_hrefs[index]
        cutpoint = room_link.index('?')
        clean_link = room_link[0:cutpoint]
        final_link = main_url + clean_link #with this link we access the full description box and can scrape it
        rooms.append(final_link)

In [66]:
print(len(rooms))

300


In [80]:
for el in rooms:
    lis.append(el)
print(len(lis))
set3 = set(lis)
print(len(set3))

600
289


In [71]:
set1 = set(rooms)
lis = obj.rooms['Luxembourg--Luxemburg']
set2 = set(lis)

In [79]:
print(set2)

{'https://www.airbnb.com/rooms/33368920', 'https://www.airbnb.com/rooms/17513394', 'https://www.airbnb.com/rooms/45148949', 'https://www.airbnb.com/rooms/25529334', 'https://www.airbnb.com/rooms/49609240', 'https://www.airbnb.com/rooms/52858571', 'https://www.airbnb.com/rooms/26095762', 'https://www.airbnb.com/rooms/21551060', 'https://www.airbnb.com/rooms/33960172', 'https://www.airbnb.com/rooms/590465895485201032', 'https://www.airbnb.com/rooms/51874649', 'https://www.airbnb.com/rooms/9100027', 'https://www.airbnb.com/rooms/49408157', 'https://www.airbnb.com/rooms/6530839', 'https://www.airbnb.com/rooms/22027001', 'https://www.airbnb.com/rooms/52894645', 'https://www.airbnb.com/rooms/45101109', 'https://www.airbnb.com/rooms/19951388', 'https://www.airbnb.com/rooms/48793599', 'https://www.airbnb.com/rooms/44476206', 'https://www.airbnb.com/rooms/580137911065831866', 'https://www.airbnb.com/rooms/51260829', 'https://www.airbnb.com/rooms/49363457', 'https://www.airbnb.com/rooms/36498202

In [72]:
print(len(set1))
print(len(lis))
print(len(set2))

223
300
229


In [50]:
print(len(obj.descriptions['Luxembourg--Luxemburg']))

300


In [51]:
count = 1
for d in obj.descriptions['Luxembourg--Luxemburg']:
    print('description room ', count, ':')
    print(d,'\n')
    count += 1

description room  1 :
a bedroom decorated with bright and sunny tones this room conveys a connection with nature and the sun providing a calm and peaceful stay as well as a wide and beautiful view over the city of luxembourg we present you the sunny and comfortable bedroom the space this room is located in bonnevoie ideally located a few minutes of walk of all amenities making your stay in luxembourg easy and enjoyable it is close to the central station 15 min walk to the city center 30 min walk and to the bus stop 2 min walk within a space of 800m you can also find restaurants cafes and shops completely furnished elegant and equipped with modern amenities our room is ideal and suitable for families couples individual travelers and business travelers we are very flexible with our guests so checkin earlier or checkout later is possible via communication a parking spot is available on request it might be already reserved so you will have to contact us in order to see if it is available 


In [52]:
for d in obj.descriptions['Luxembourg--Luxemburg']:
    if d == '':
        print('Alarm')

In [54]:
print(obj.descriptions['Luxembourg--Luxemburg'][20])
print(obj.descriptions['Luxembourg--Luxemburg'][5])

luxembourg city cessange near cloche dor 1520 min by walk to reach main places restaurants animated areas shops 2 minutes walk from the bus exchange with direct bus to luxembourg train station 8 minutes clean 1 single bed and 1 convertible sofa well equiped kitchen smart tv no cable free parking available on weekends
luxembourg city cessange near cloche dor 1520 min by walk to reach main places restaurants animated areas shops 2 minutes walk from the bus exchange with direct bus to luxembourg train station 8 minutes clean 1 single bed and 1 convertible sofa well equiped kitchen smart tv no cable free parking available on weekends


In [73]:
print(obj.rooms['Luxembourg--Luxemburg'][21])
print(obj.rooms['Luxembourg--Luxemburg'][1])

https://www.airbnb.com/rooms/29258164
https://www.airbnb.com/rooms/29258164


In [74]:
print(obj.rooms['Luxembourg--Luxemburg'][21] == obj.rooms['Luxembourg--Luxemburg'][1])

True


In [53]:
for d in range(len(obj.descriptions['Luxembourg--Luxemburg'])):
    totest = obj.descriptions['Luxembourg--Luxemburg'][d]
    for k in obj.descriptions['Luxembourg--Luxemburg']:
        if obj.descriptions['Luxembourg--Luxemburg'].index(k) != d:
            if k == totest:
                print('double at:', d, 'and', obj.descriptions['Luxembourg--Luxemburg'].index(k))
        

double at: 20 and 5
double at: 20 and 5
double at: 21 and 1
double at: 21 and 1
double at: 24 and 6
double at: 24 and 6
double at: 26 and 10
double at: 26 and 10
double at: 28 and 13
double at: 28 and 13
double at: 29 and 8
double at: 29 and 8
double at: 30 and 14
double at: 30 and 14
double at: 33 and 12
double at: 33 and 12
double at: 51 and 38
double at: 51 and 38
double at: 52 and 25
double at: 52 and 25
double at: 61 and 19
double at: 61 and 19
double at: 66 and 40
double at: 66 and 40
double at: 69 and 46
double at: 69 and 46
double at: 69 and 46
double at: 70 and 50
double at: 70 and 50
double at: 72 and 53
double at: 72 and 53
double at: 72 and 53
double at: 72 and 53
double at: 79 and 55
double at: 79 and 55
double at: 82 and 18
double at: 82 and 18
double at: 92 and 49
double at: 92 and 49
double at: 93 and 41
double at: 93 and 41
double at: 99 and 46
double at: 99 and 46
double at: 99 and 46
double at: 100 and 62
double at: 100 and 62
double at: 104 and 22
double at: 104 and

In [351]:
print(obj.rooms['Luxembourg--Luxemburg'][109])

https://www.airbnb.com/rooms/43803257


In [221]:
# "ll4r2nl dir dir-ltr" class finds the text embedded in html without box
# continue: find proper class to scrape the description box
# <div class="_1jza0fl">
# <div class="_178zopn">
# <div class="c1yo0219 dir dir-ltr"
# getting the class <div class="_178zopn"> does not work, class <div class="c1yo0219 dir dir-ltr" refers to a lot of other info as well 
# if translated in position 9, otherwise position 8
# getting the information from the box does not work

In [222]:
# <div class="_1a5fl1v">
# <div class="_3hmsj">
# <div class="_1t0jy5tw"></div>

In [55]:
start_html = requests.get('https://www.airbnb.com/s/Luxembourg--Luxemburg/homes')
start_soup = BeautifulSoup(start_html.content, 'html.parser')

In [42]:
links = start_soup.find_all(href = True)
sub = 'room'
links_fin = []
for link in links:
    l = link['href']
    if sub in l:
        cut = l.find('?')
        final = main_url + l[:cut]
        if final not in links_fin:
            links_fin.append(final)
print(links_fin)

['https://www.airbnb.com/rooms/51343823', 'https://www.airbnb.com/rooms/652717898285340648', 'https://www.airbnb.com/rooms/667242569001076835', 'https://www.airbnb.com/rooms/48325095', 'https://www.airbnb.com/rooms/612627051716247514', 'https://www.airbnb.com/rooms/651896822687140509', 'https://www.airbnb.com/rooms/50367192', 'https://www.airbnb.com/rooms/665191089083904332', 'https://www.airbnb.com/rooms/25532722', 'https://www.airbnb.com/rooms/54203432', 'https://www.airbnb.com/rooms/48601427', 'https://www.airbnb.com/rooms/566456384680673313', 'https://www.airbnb.com/rooms/567371485303188402', 'https://www.airbnb.com/rooms/22392206', 'https://www.airbnb.com/rooms/658533985104384949', 'https://www.airbnb.com/rooms/24265205', 'https://www.airbnb.com/rooms/633342370273244337', 'https://www.airbnb.com/rooms/39339443', 'https://www.airbnb.com/rooms/29167908', 'https://www.airbnb.com/rooms/633074396962786913']


In [36]:
test_link = links_fin[0]

In [37]:
test_link.find('?')

15

In [38]:
test_link[:15]

'/rooms/51343823'

In [43]:
print(len(links_fin))

20


In [ ]:
#<h1 tabindex="-1" class="_14i3z6h" 

In [ ]:
#<div class="_le6wlg">

In [ ]:
#div class="c1yo0219 dir dir-ltr"

In [311]:
test_html = requests.get('https://www.airbnb.com/rooms/51343823')
test_html2 = requests.get('https://www.airbnb.com/rooms/652717898285340648')
test_html3 = requests.get('https://www.airbnb.com/rooms/612627051716247514')
test_html4 = requests.get('https://www.airbnb.com/rooms/48325095')
test_html5 = requests.get('https://www.airbnb.com/rooms/651896822687140509')
test_html6 = requests.get('https://www.airbnb.com/rooms/649348542190748049')
soup = BeautifulSoup(test_html.content, 'html.parser')
soup2 = BeautifulSoup(test_html2.content, 'html.parser')
soup3 = BeautifulSoup(test_html3.content, 'html.parser')
soup4 = BeautifulSoup(test_html4.content, 'html.parser')
soup5 = BeautifulSoup(test_html5.content, 'html.parser')
soup6 = BeautifulSoup(test_html6.content, 'html.parser')

In [312]:
text1 = soup.find_all(class_ = 'c1yo0219 dir dir-ltr')
text2 = soup2.find_all(class_ = 'c1yo0219 dir dir-ltr')
text3 = soup3.find_all(class_ = 'c1yo0219 dir dir-ltr')
text4 = soup4.find_all(class_ = 'c1yo0219 dir dir-ltr')
text5 = soup5.find_all(class_ = 'c1yo0219 dir dir-ltr')
text6 = soup6.find_all(class_ = 'c1yo0219 dir dir-ltr')

In [313]:
print(len(text1))
print(len(text2))
print(len(text3))
print(len(text4))
print(len(text5))
print(len(text6))

21
21
22
20
21
20


In [263]:
text6[6].get_text('|')

'Entire rental unit hosted by Arca Properties\xa0Luxembourg|2 guests| · |1 bedroom| · |1 bed| · |1 bathroom'

In [133]:
for i in range(len(text4)):
    print(i)
    print(text1[i].get_text('|'))
    print(text2[i].get_text('|'))
    print(text3[i].get_text('|'))
    print(text4[i].get_text('|') + '\n')
print(20)
print(text1[20].get_text('|'))
print(text2[20].get_text('|'))
print(text3[20].get_text('|') + '\n')
print(21)
print(text3[21].get_text('|'))

0
Become a host|Sign up|Log in|Host your home|Host an experience|Help
Become a host|Sign up|Log in|Host your home|Host an experience|Help
Become a host|Sign up|Log in|Host your home|Host an experience|Help
Become a host|Sign up|Log in|Host your home|Host an experience|Help

1
ONE PRIVATE BEDROOM Near Luxembourg Gare|4.94 ·|18 reviews|·|Luxembourg, District de Luxembourg, Luxembourg|Share|Save
Cozy Studio at place de paris Lux (Quiet Location)|2 reviews|·|Luxembourg, Luxembourg|Share|Save
THE N.D Appartement in the middle of Luxembourg.|4.80 ·|10 reviews|·|󰀃|Superhost|·|Luxembourg, Luxembourg|Share|Save
Welcome by Erny and Augusto|5.0 ·|11 reviews|·|Bartringen, Distrikt Luxemburg, Luxembourg|Share|Save

2
Show all photos
Show all photos
Show all photos
Show all photos

3
Photos|Amenities|Reviews|Location
Photos|Amenities|Reviews|Location
Photos|Amenities|Reviews|Location
Photos|Amenities|Reviews|Location

4
loading|Check availability| 
loading|Check availability| 
loading|Check availabi

In [138]:
#way to get description from text collection
hint_desc1 = 'Every booking includes free protection'
hint_desc2 = 'Some info has been automatically translated'
hint_nodesc1 = "Where you’ll be"
hint_nodesc2 = 'What this place offers'
hint_nodesc2 = "Where you'll sleep"
for i in range(6,len(text4)):
    te = text4[i].get_text()
    if hint_desc1 in te:
        if hint_nodesc1 or hint_nodesc2 or hint_nodesc3 in text4[i+1].get_text(' '):
            print(i)
            print('no desc')
        elif hint_desc2 in text4[i+1].get_text(' '):
            desc = text4[i+2].get_text(' ').lower()
            desc_noemo = emoji.demojize(desc)
            desc_fin = re.sub(r':[a-z_]+:','', desc_noemo)
            clean = " ".join(desc_fin.split())
        else:
            desc = text4[i+1].get_text('|').lower()
            desc_noemo = emoji.demojize(desc)
            desc_fin = re.sub(r':[a-z_]+:','', desc_noemo)
            clean = " ".join(desc_fin.split())
#print(clean)

8
no desc


In [317]:
hint_desc1 = 'Every booking includes free protection'
hint_desc2 = 'Some info has been automatically translated'
hint_nodesc1 = "Where you’ll be"
hint_nodesc2 = 'What this place offers'
hint_nodesc3 = "Where you'll sleep"
#description = []
for i in range(6,len(text6)):
    text_desc = text6[i].get_text()
    if hint_desc1 in text_desc:
        check_next = text6[i+1].get_text(' ')
        print(check_next)
        if hint_nodesc1 in check_next or hint_nodesc2 in check_next or hint_nodesc3 in check_next:
            print(check_next)
            description.append('NaN')
            print(hint_nodesc1 in check_next)
            print(hint_nodesc2 in check_next)
            print(hint_nodesc3 in check_next)
        elif hint_desc2 in text6[i+1].get_text(' '):
            desc = text6[i+2].get_text(' ').lower()
            desc_noemo = emoji.demojize(desc)
            desc_fin = re.sub(r':[a-z_]+:','', desc_noemo)
            clean = " ".join(desc_fin.split())
            description = clean
        else:
            desc = text6[i+1].get_text('|').lower()
            desc_noemo = emoji.demojize(desc)
            desc_fin = re.sub(r':[a-z_]+:','', desc_noemo)
            clean = " ".join(desc_fin.split())
            description = clean

Some info has been automatically translated.   Show original language


In [318]:
print(description)

nice, very bright, quiet apartment with a terrace . separate kitchen and spacious living room


In [328]:
#get title
header = text6[1].get_text('|', strip = True)
cut = header.find('|')
title = header[:cut].lower().replace('\n', ' ').strip()
print(title)

luxembourg gasperich: very bright apartment


In [325]:
text6[1].get_text('|', strip = True)

'Luxembourg Gasperich:\nvery bright apartment|2 reviews|·|Luxembourg, Luxembourg|Share|Save'

In [151]:
#get type and host
subheader = text1[6].get_text('|')
cut = subheader.find('|')
type_host = subheader[:cut].lower().split()
pos_host = type_host.index('by')
host = ' '.join(type_host[pos_host + 1:])
type_ = ' '.join(type_host[0:2])
print(type_host)
print(pos_host)
print(host)
print(type_)

['private', 'room', 'in', 'apartment', 'hosted', 'by', 'edoh']
5
edoh
private room
4


In [ ]:
doc = nlp(host.get_text())
if doc.ents != ():
ent = doc.ents[0].text
self.host[city].append(ent)
else:
self.host[city].append('NaN')

In [269]:
#location
#address always NaN
# can bestrcutred with length 2 --> city and country or length 3 --> city, state, country
address = []
district = []
city = []
country = []
hint_loc = "Where you’ll be"
for i in range(6,len(text1)):
    te = text1[i].get_text('|')
    if hint_loc in te:
        loc = te.split('|')
        loc_ = loc[1].lower()
        address.append(loc_)
        loc_split = loc_.split()
        if len(loc_split) == 2:
            city.append(loc_split[0])
            country.append(loc_split[1])
        elif len(loc_split) == 3:
            city.append(loc_split[0])
            country.append(loc_split[2])
            
        if len(loc) == 2:
            district.append('NaN')
        else:
            output = ''
            cut = loc.index('Show more')
            loc_desc = loc[2:cut]
            for ele in loc_desc:
                output += ele
            doc = nlp(output)
            print(doc.ents)
            ents = []
            for ent in doc.ents:
                if str(ent).lower() not in loc_:
                    if ent.label_ == 'GPE' or ent.label_ == 'LOC':
                        ents.append(str(ent).lower())
            print(ents)
            if ents == []:
                district.append('NaN')
            else:
                counts = []
                for ent in ents:
                    ent_c = output.count(ent)
                    counts.append(ent_c)
                pos = counts.index(max(counts))
                district.append(ents[pos])
print(loc)
print(loc_)
print(address)
print(district)
print(city)
print(country)

['Where you’ll be', 'Luxembourg, District de Luxembourg, Luxembourg']
luxembourg, district de luxembourg, luxembourg
['luxembourg, district de luxembourg, luxembourg']
['NaN']
[]
[]


In [271]:
loc_.split(',')

['luxembourg', ' district de luxembourg', ' luxembourg']

In [199]:
te = text5[16].get_text('|')
loc = te.split('|')
loc_ = loc[1].lower()

In [203]:
output = ''
cut = loc.index('Show more')
loc_desc = loc[2:cut]
for ele in loc_desc:
    output += ele
doc = nlp(output)

In [206]:
for ent in doc.ents:
    print(ent.text,  ent.label_)

the Gasperich District LOC
Luxembourg GPE
Luxembourg GPE
20-min QUANTITY
15 CARDINAL
The Old Quarter LOC
UNESCO ORG
Le Chemin de la Corniche FAC
Notre Dame Cathedral3 FAC
Pfaffenthal Lift FAC
4 CARDINAL
Mondorf-les-Bains FAC
National Museum of History and Art6 ORG
Grand Duke Jean Museum of Modern Art ORG
MUDAM ORG
Vianden Castle FAC
Vianden GPE
one CARDINAL
Palais Grand-Ducal FAC
Parc Merveilleux10 FAC
Bock Cliff LOC
Mullerthal Trail FAC
Echternach GPE
Benedictine Abbey FAC
13 CARDINAL
Bourscheid Castle15.Rockhal FAC
Esch-sur-AlzetteHere GPE
Cloche d’Or shopping FAC
Restaurant Bubblies- ORG


In [202]:
print(loc)

['Where you’ll be', 'Luxembourg, Luxembourg', 'The apartment is located in the Gasperich District in Luxembourg. It’s a peaceful area and you’ll find easy public transport options to Luxembourg city center, about a 20-min journey by train.', 'In addition, here’s a list of the 15 MOST VISITED ATTRACTIONS in Luxembourg:', '1. The Old Quarter in Luxembourg City (a UNESCO World Heritage Site), for example, the historic neighborhoods of Le Chemin de la Corniche and Grund.', '2. Notre Dame Cathedral', '3. Pfaffenthal Lift ', '4. Mondorf-les-Bains (home of the best thermal springs in all of Europe!)', '5. National Museum of History and Art', '6. Grand Duke Jean Museum of Modern Art or MUDAM (incredible collection of contemporary art & exhibits, sculptures, architecture, and more!)', '7. Vianden Castle and Vianden town (one of the most scenic towns in all of Luxembourg)', '8. Palais Grand-Ducal (Palace Residence of the Grand Duke of Luxembourg)', '9. Parc Merveilleux', '10. Bock Cliff & City C

In [ ]:
doc = nlp(output)

In [165]:
print(output)

Close to bars and good restaurants,  you are ideally located to visit Luxembourg. Access to the city centre is also possible on foot.


In [170]:
doc = nlp(output)

In [179]:
ents = []
for el in doc.ents:
    print(str(el))

Luxembourg


In [100]:
text3[10].get_text()

'Very well located apartment, in the city center with a quiet attenuating. 300 m from the train station of Luxembourg 600 m to Luxembourg city center 50 m bus line 150 m from Luxembourg tram The apartment is very central you can benefit from public transportation, which are free in Luxembourg.You have the chance to park for free near the apartment or you can also enjoy secure and paid parking. I hope you enjoy your stay in ND. Apartment.The spaceThe accommodation is warm and enjoys incredible light.'

In [80]:
for t in text1:
    print(t.get_text(strip = True) + '\n')

Become a hostSign upLog inHost your homeHost an experienceHelp

ONE PRIVATE BEDROOM Near Luxembourg Gare4.94 ·18 reviews·Luxembourg, District de Luxembourg, LuxembourgShareSave

Show all photos

PhotosAmenitiesReviewsLocation

loadingCheck availability

Private room in apartment hosted by Edoh1 guest·1 bedroom·1 bed·1 shared bathroomDedicated workspaceA private room with wifi that’s well suited for working.Great check-in experience94% of recent guests gave the check-in process a 5-star rating.Every booking includes free protection from Host cancellations, listing inaccuracies, and other issues like trouble checking in.Learn moreEnjoy a stylish experience at this centrally-located private bedroom in a 2 bedroom apartment with easy access to Luxembourg free public transport, City Centre, shops and supermarkets.Please note that the room is for 1 guest only.Our accommodation is for guests on short-stay or trips only and cannot be used by guest to register as their permanent address in Luxe

In [81]:
for t in text2:
    print(t.get_text(strip = True) + '\n')

Become a hostSign upLog inHost your homeHost an experienceHelp

Cozy Studio at place de paris Lux (Quiet Location)2 reviews·Luxembourg, LuxembourgShareSave

Show all photos

PhotosAmenitiesReviewsLocation

loadingCheck availability

Entire rental unit hosted by Marco1 guest·1 bedroom·1 bed·1 bathroomSelf check-inCheck yourself in with the lockbox.Every booking includes free protection from Host cancellations, listing inaccuracies, and other issues like trouble checking in.Learn moreSome info has been automatically translated.Show original languageThe studio is for 1 person and is located on the 2 floor, number 4.The spaceWith private bathroom, kitchenette, free WiFi, smart TV (Netflix, Youtube, etc.), cleaning and linen change once a week. Washing machine on the ground floor.What this place offersKitchenWifiTVPaid washer – In buildingLuggage drop-off allowedHair dryerZanker refrigeratorMicrowaveLong-term stays allowedUnavailable: Carbon monoxide alarmCarbon monoxide alarmShow all 36 am

In [ ]:
<div class="c1yo0219 dir dir-ltr"><div style="--gp-section-max-width:1120px;"><div class="_npr0qi" style="border-top-color: rgb(221, 221, 221);"></div><div data-plugin-in-point-id="DESCRIPTION_DEFAULT" data-section-id="DESCRIPTION_DEFAULT" style="padding-top: 32px; padding-bottom: 48px;"><div style="line-height: 24px;"><span><span class="ll4r2nl dir dir-ltr">Enjoy a stylish experience at this centrally-located private bedroom in a 2 bedroom apartment with easy access to Luxembourg free public transport, City Centre, shops and supermarkets.<br><br>Please note that the room is for 1 guest only.<br><br>Our accommodation is for guests on short-stay or trips only and cannot be used by guest to register as their permanent address in Luxembourg.</span></span></div></div></div></div>

In [5]:
test = soup.find_all(text = True)

In [34]:
blacklist = ['[document]', 'noscript', 'header', 'html', 'meta', 'head', 'input', 'script', 'title']

In [35]:
output = []
for t in test:
    if t.parent.name not in blacklist:
        if t != '\n':
            output.append(t)
print(output)

['Skip to content', 'We’re sorry, some parts of the Airbnb website don’t work properly without JavaScript enabled.', '._1a5fl1v{--dls19-brand-color:#FF385C !important;--dls19-ui-brand-color:#222222 !important;--dls19-pdp-listing-title-weight:600 !important;--dls19-pdp-listing-title-size:26px !important;--dls19-pdp-listing-title-line-height:30px !important;--dls19-brand-gradient:-webkit-linear-gradient(to right, #E61E4D 0%, #E31C5F 50%, #D70466 100%) !important;--dls19-brand-gradient:-moz-linear-gradient(to right, #E61E4D 0%, #E31C5F 50%, #D70466 100%) !important;--dls19-brand-gradient:linear-gradient(to right, #E61E4D 0%, #E31C5F 50%, #D70466 100%) !important;--dls19-brand-gradient-radial:-webkit-radial-gradient(circle at center,\n  #FF385C 0%,\n  #E61E4D 27.5%,\n  #E31C5F 40%,\n  #D70466 57.5%,\n  #BD1E59 75%,\n  #BD1E59 100%\n) !important;--dls19-brand-gradient-radial:-moz-radial-gradient(circle at center,\n  #FF385C 0%,\n  #E61E4D 27.5%,\n  #E31C5F 40%,\n  #D70466 57.5%,\n  #BD1E59 

In [ ]:
<button class="cley8gd r1qu77pt f1s3caxj dir dir-ltr" aria-label="Restaurant: Chiche!" data-veloute="map/markers/MicroCircleMarker"><div class="cv55pes dir dir-ltr"><div class="cfxcvek pgtll9r dir dir-ltr" style="--pin-background-color:#FF7D61;"><div class="c5i5p7w dir dir-ltr"><svg viewBox="0 0 32 32" xmlns="http://www.w3.org/2000/svg" aria-hidden="true" role="presentation" focusable="false" style="display: block; height: 10px; width: 10px; fill: currentcolor;"><path d="M23 1a5 5 0 0 1 5 5v4a5.002 5.002 0 0 1-3.5 4.771V31h-3V14.771A5.002 5.002 0 0 1 18 10V6a5 5 0 0 1 5-5zM4 1h2v6h2V1h2v6h2V1h2v9a5.002 5.002 0 0 1-3.5 4.771V31h-3V14.77a5.014 5.014 0 0 1-3.495-4.546L4 10V1z"></path></svg></div></div></div><div class="_1ldlefdw" style="left: calc(100% + 6px); top: calc(50% - 0px); transform: translate(0px, -50%); color: var(--f-k-smk-x);"><div class="_hoais0r" style="cursor: pointer;">Chiche!</div></div></button>

In [ ]:
<button class="cley8gd r1qu77pt f1s3caxj dir dir-ltr" aria-label="History museum: Bock Casemates" data-veloute="map/markers/MicroCircleMarker"><div class="cv55pes dir dir-ltr"><div class="cfxcvek pgtll9r dir dir-ltr" style="--pin-background-color:#68A1F9;"><div class="c5i5p7w dir dir-ltr"><svg viewBox="0 0 32 32" xmlns="http://www.w3.org/2000/svg" aria-hidden="true" role="presentation" focusable="false" style="display: block; height: 10px; width: 10px; fill: currentcolor;"><path d="M26 1a5 5 0 0 1 0 10v18a2 2 0 0 1-1.85 1.995L24 31H8a2 2 0 0 1-2-2V11A5 5 0 1 1 6 1zm-2.9 9.5H9V28h2.6V13.1h3.1V28h2.6V13.1h3.2V28h2.6V10.5zM6 4.5a1.5 1.5 0 1 0 0 3 1.5 1.5 0 0 0 0-3zm20 0a1.5 1.5 0 1 0 0 3 1.5 1.5 0 0 0 0-3z"></path></svg></div></div></div><div class="_1ldlefdw" style="left: calc(100% + 6px); top: calc(50% - 0px); transform: translate(0px, -50%); color: var(--f-k-smk-x);"><div class="_hoais0r" style="cursor: pointer;">Bock Casemates</div></div></button>

In [9]:
test3 = soup.find_all(class_ = '_10u42wu')
print(test3)

[<div class="_10u42wu"><span aria-busy="true" style="display:inline-block;height:100%;width:100%"><span class="_8kplbn" style="height:100%;width:100%"></span></span></div>]


In [29]:
output2 = []
for t in test:
    if t != '\n':
        output2.append(t)
output == output2

False

In [30]:
len(output2)

221

In [6]:
test

['html',
 '\n',
 "(function() {\n  var pgRequest = new XMLHttpRequest();\n  var diffStamp = Date.now().toString() + Math.random().toString().substring(2);\n  pgRequest.open('GET', '/pg_pixel?r=' + encodeURIComponent(document.referrer || '') + '&diff=' + diffStamp, true);\n  pgRequest.send();\n})()",
 '(function(){function a(){return"UnknownPageName"}function b(){return"unknown"}function c(){var a,b,c,d;return null!==(a=null===(b=navigator)||void 0===b||null===(c=b.serviceWorker)||void 0===c||null===(d=c.controller)||void 0===d?void 0:d.scriptURL)&&void 0!==a?a:"none"}window._bufferedErrors=[];const d=window.onerror;window.onerror=(a,b,c,e,f)=>{d&&d.call(window,a,b,c,e,f),window._bufferedErrors&&window._bufferedErrors.push([a,b,c,e,f])},window.addEventListener("load",()=>{if(!window._errorReportingInitialized){const g=window._bufferedErrors||[];if(window.fetch&&0<g.length){var d,e,f;window.fetch("https://notify.bugsnag.com/",{body:JSON.stringify({apiKey:"e393bc25e52fe915ffb56c14ddf2ff1b

In [36]:
parent_names = []
for t in test:
    if t.parent.name not in blacklist:
        if t != '\n':
            a = t.parent.name
            parent_names.append(a)
print(parent_names)

['a', 'div', 'style', 'div', 'div', 'div', 'div', 'div', 'div', 'h1', 'span', 'button', 'span', 'span', 'div', 'div', 'div', 'div', 'div', 'div', 'div', 'span', 'span', 'span', 'h2', 'span', 'span', 'span', 'span', 'span', 'span', 'span', 'div', 'div', 'div', 'div', 'div', 'button', 'span', 'span', 'span', 'h2', 'div', 'div', 'h2', 'div', 'div', 'div', 'div', 'div', 'div', 'div', 'div', 'div', 'div', 'button', 'h2', 'div', 'span', 'span', 'div', 'div', 'div', 'div', 'div', 'span', 'span', 'span', 'span', 'div', 'span', 'span', 'div', 'span', 'div', 'span', 'div', 'span', 'div', 'span', 'div', 'span', 'div', 'span', 'button', 'h2', 'div', 'h2', 'li', 'span', 'button', 'span', 'span', 'li', 'li', 'li', 'span', 'li', 'li', 'li', 'span', 'li', 'li', 'li', 'span', 'a', 'div', 'h2', 'h3', 'span', 'span', 'span', 'span', 'span', 'span', 'h3', 'span', 'span', 'span', 'span', 'span', 'h3', 'span', 'h2', 'span', 'span', 'span', 'span', 'span', 'span', 'span', 'span', 'span', 'span', 'span', 'spa

In [21]:
test2 = soup.find_all(class_="ll4r2nl dir dir-ltr")

In [22]:
text = []
for t in test2:
    text.append(t.get_text())

In [23]:
text

['Enjoy a stylish experience at this centrally-located private bedroom in a 2 bedroom apartment with easy access to Luxembourg free public transport, City Centre, shops and supermarkets.Please note that the room is for 1 guest only.Our accommodation is for guests on short-stay or trips only and cannot be used by guest to register as their permanent address in Luxembourg.',
 'English',
 '100%',
 'within an hour']

In [327]:
z = soup.find_all(class_ = "col-md-5 col-middle") #this class embedds error text if a link should lead to an error

In [326]:
for u in z:
    print(u.get_text())


Oops!
We can't seem to find the page you're looking for.
Error code: 404

Here are some helpful links instead:
Home
Search
Help
Travelling on Airbnb
Hosting on Airbnb
Trust & Safety




In [321]:
mytxt= open('soup.txt','w')
mytxt.write(str(soup))
mytxt.close()

In [44]:
test_html = requests.get('https://www.airbnb.com/rooms/51343823?source_impression_id=p3_1657615247_WlQIjM7ww3rsbKk%2F')
test_html2 = requests.get('https://www.airbnb.com/rooms/652717898285340648?source_impression_id=p3_1657615253_VmR2C3u04keOzT6D')
#test_html3 = requests.get('https://www.airbnb.com/rooms/39788619?source_impression_id=p3_1652255969_MAD3h08Z1t4J2WF9')
soup = BeautifulSoup(test_html.content, 'html.parser')
soup2 = BeautifulSoup(test_html2.content, 'html.parser')
#soup3 = BeautifulSoup(test_html3.content, 'html.parser')

In [45]:
te=soup.find_all(class_ = "ll4r2nl dir dir-ltr")
te2=soup2.find_all(class_ = "ll4r2nl dir dir-ltr")
#te3=soup3.find_all(class_ = "ll4r2nl dir dir-ltr")

In [53]:
print(te2)

[<span class="ll4r2nl dir dir-ltr">Some info has been automatically translated.</span>, <span class="ll4r2nl dir dir-ltr">The studio is for 1 person and is located on the 2 floor, number 4.<br/><br/><span class="_1di55y9">The space</span><br/>With private bathroom, kitchenette, free WiFi, smart TV (Netflix, Youtube, etc.), cleaning and linen change once a week. Washing machine on the ground floor.</span>, <span class="ll4r2nl dir dir-ltr">English, Français, Deutsch</span>, <span class="ll4r2nl dir dir-ltr">100%</span>, <span class="ll4r2nl dir dir-ltr">within an hour</span>]


In [47]:
print(te[0].get_text())
print(te2[0].get_text())
#print(te3[0].get_text())

Enjoy a stylish experience at this centrally-located private bedroom in a 2 bedroom apartment with easy access to Luxembourg free public transport, City Centre, shops and supermarkets.Please note that the room is for 1 guest only.Our accommodation is for guests on short-stay or trips only and cannot be used by guest to register as their permanent address in Luxembourg.
Some info has been automatically translated.


In [48]:
if te[0].get_text() == 'Some info has been automatically translated.':
    print(True)
if te2[0].get_text() == 'Some info has been automatically translated.':
    print(True)
#if te3[0].get_text() == 'Some info has been automatically translated.':
    #print(True)

True


In [51]:
test=te[0].get_text(separator = ' ')

In [52]:
test = " ".join(test.split()).lower()
print(test)

enjoy a stylish experience at this centrally-located private bedroom in a 2 bedroom apartment with easy access to luxembourg free public transport, city centre, shops and supermarkets. please note that the room is for 1 guest only. our accommodation is for guests on short-stay or trips only and cannot be used by guest to register as their permanent address in luxembourg.


In [259]:
test = test.lower()
print(test)

this ideally located kozi room, gives access to all strategic, commercial and tourist places. close to all means of transport train station, airport, trams, bus, you can take advantage of the free transport to get there. the space bedroom 1 is in an apartment on the 1st floor. - 3 rooms available : bedroom, bathroom, toilet with sink - no kitchen available ! - on the other hand, the room is equipped with a microwave, coffee machine, boiler, iron and ironing board, as well as sheets and towels.


In [303]:
test_ = te2[0].get_text()

In [304]:
test_ = emoji.demojize(test_)

In [305]:
test_ = re.sub(r':[a-z_]+:','',test_)

In [306]:
test_ = re.sub(r'[^\w\s]', '', test_)

In [307]:
test_ = " ".join(test_.split()).lower()

In [308]:
print(test_)

fully equipped and furnished studio located in the heart of luxembourg city with a unique location on the 4th floor this apartment offers easy access to grocery stores shops restaurants bars and not only the new luxembourgish tram is around the corner and all the city buses are a few minutes away you are super central located and can do everything within walking distance the studio has fresh bed linens 247 concierge shampoo coffee maker microwave washing machine smart tv big windows and so on we care about every single guest that we hostthe spacethe studio gives you an openspace feeling it has one room which includes the bedroom and kitchenthe place is very well furnished it includes fresh bedsheets linens towels cleaning products free wifi refrigerator cookware and not only


In [210]:
print(len(te))

7


In [166]:
print(te[8].get_text())

Fully equipped and furnished Studio located in the heart of Luxembourg city 🏙️. With a unique location on the 4th Floor, this apartment offers easy access to Grocery Stores, Shops, Restaurants, Bars, and not only. The new Luxembourgish tram 🚋 is around the corner and all the city buses are a few minutes away... You are super central located and can do everything within walking distance! The Studio 🏠 has: fresh bed linens, 24/7 concierge, shampoo, coffee maker, microwave, washing machine, smart TV, big windows , and so on... ✨. We care about every single guest that we host!The spaceThe Studio🏠 gives you an open-space feeling; it has one room which includes the Bedroom🛏️ and Kitchen🍳.The place is very well furnished. It includes fresh bedsheets, linens, towels, cleaning products, free Wi-fi, refrigerator, cookware and not only.Show more


In [170]:
text = []
for t in te:
    t.find_all(class_ = "_178zopn")
    text.append(t)
print(text)

[<div class="c1yo0219 dir dir-ltr"><header class="_1d8im2uw"><div class="_z5inxef"><div class="_otdka"><a aria-label="Airbnb Homepage" class="_1f41ax74" href="/"><div class="_1grlqto"><svg fill="currentcolor" height="32" style="display:block" width="102"><path d="M29.24 22.68c-.16-.39-.31-.8-.47-1.15l-.74-1.67-.03-.03c-2.2-4.8-4.55-9.68-7.04-14.48l-.1-.2c-.25-.47-.5-.99-.76-1.47-.32-.57-.63-1.18-1.14-1.76a5.3 5.3 0 00-8.2 0c-.47.58-.82 1.19-1.14 1.76-.25.52-.5 1.03-.76 1.5l-.1.2c-2.45 4.8-4.84 9.68-7.04 14.48l-.06.06c-.22.52-.48 1.06-.73 1.64-.16.35-.32.73-.48 1.15a6.8 6.8 0 007.2 9.23 8.38 8.38 0 003.18-1.1c1.3-.73 2.55-1.79 3.95-3.32 1.4 1.53 2.68 2.59 3.95 3.33A8.38 8.38 0 0022.75 32a6.79 6.79 0 006.75-5.83 5.94 5.94 0 00-.26-3.5zm-14.36 1.66c-1.72-2.2-2.84-4.22-3.22-5.95a5.2 5.2 0 01-.1-1.96c.07-.51.26-.96.52-1.34.6-.87 1.65-1.41 2.8-1.41a3.3 3.3 0 012.8 1.4c.26.4.45.84.51 1.35.1.58.06 1.25-.1 1.96-.38 1.7-1.5 3.74-3.21 5.95zm12.74 1.48a4.76 4.76 0 01-2.9 3.75c-.76.32-1.6.41-2.42.3

In [169]:
print(soup)

<!DOCTYPE html>

<html class="scrollbar-gutter" data-is-hyperloop="true" dir="ltr" lang="en-IE"><head><meta charset="utf-8"/><meta content="en-IE" name="locale"/><meta content="notranslate" name="google"/><meta content="authenticity_token" id="csrf-param-meta-tag" name="csrf-param"/><meta content="" id="csrf-token-meta-tag" name="csrf-token"/><meta content="" id="english-canonical-url"/><meta content="on" name="twitter:widgets:csp"/><meta content="yes" name="mobile-web-app-capable"/><meta content="yes" name="apple-mobile-web-app-capable"/><meta content="Airbnb" name="application-name"/><meta content="Airbnb" name="apple-mobile-web-app-title"/><meta content="#ffffff" name="theme-color"/><meta content="#ffffff" name="msapplication-navbutton-color"/><meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/><meta content="/?utm_source=homescreen" name="msapplication-starturl"/><script>(function() {
  var pgRequest = new XMLHttpRequest();
  var diffStamp = Date.now().t

In [118]:
texts = []
for t in te:
    text = t.get_text()
    texts.append(text)
print(texts)

['Some info has been automatically translated.', 'This ideally located Kozi room, gives access to all strategic, commercial and tourist places.  Close to all means of transport train station, airport, trams, bus, you can take advantage of the free transport to get there.The spaceBedroom 1 is in an apartment on the 1st floor.- 3 rooms available : bedroom, bathroom, toilet with sink- No kitchen available !- On the other hand, the room is equipped with a microwave, coffee machine, boiler, iron and ironing board, as well as sheets and towels.', 'Very convenient neighborhood, close to public transport, close to the city center, 1 km from the train station, with shops, bakeries, restaurants nearby', 'If you need me, you can always contact me by phone or on the Airbnb site', 'English, Français', '100%', 'within an hour']


In [ ]:
<span class="ll4r2nl dir dir-ltr">This ideally located Kozi room, gives access to all strategic, commercial and tourist places.  Close to all means of transport train station, airport, trams, bus, you can take advantage of the free transport to get there.<br><br><span class="_1di55y9">The space</span><br>Bedroom 1 is in an apartment on the 1st floor.<br>- 3 rooms available : bedroom, bathroom, toilet with sink<br>- No kitchen available !<br>- On the other hand, the room is equipped with a microwave, coffee machine, boiler, iron and ironing board, as well as sheets and towels.</span>

In [126]:
print(obj.rooms)

{'Luxembourg--Luxemburg': ['https://www.airbnb.com/rooms/39788619?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/48750286?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/17513394?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/565783019919522564?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/615578251032785787?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/51629605?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/49408157?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/609981955471356723?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/45671610?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/576629623395968737?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/46920014?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/27420422?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/594983118913142540?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/29167908?&modal=DESCRIPTION', 'https://www.airbnb.com/rooms/51343823?&modal=DESCRIPTION', 'https://www.airbnb.com

In [61]:
cities_url = {}
cities_url['Munich'] = ['test1', 'test3']
cities_url['Berlin'] = ['test2']

In [62]:
print(cities_url['Munich'])

['test1', 'test3']


In [63]:
for i in cities_url['Munich']:
    print(i)

test1
test3
